In [2]:
import pandas as pd
import numpy as np
file_path = 'data/titanic/train.csv'
data = pd.read_csv(file_path)
data.dropna(inplace=True)
y = data['Survived'].values
print(data.head())
print(data.dtypes)

    PassengerId  Survived  Pclass  \
1             2         1       1   
3             4         1       1   
6             7         0       1   
10           11         1       3   
11           12         1       1   

                                                 Name     Sex   Age  SibSp  \
1   Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
3        Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
6                             McCarthy, Mr. Timothy J    male  54.0      0   
10                    Sandstrom, Miss. Marguerite Rut  female   4.0      1   
11                           Bonnell, Miss. Elizabeth  female  58.0      0   

    Parch    Ticket     Fare Cabin Embarked  
1       0  PC 17599  71.2833   C85        C  
3       0    113803  53.1000  C123        S  
6       0     17463  51.8625   E46        S  
10      1   PP 9549  16.7000    G6        S  
11      0    113783  26.5500  C103        S  
PassengerId      int64
Survived 

## Pre Processing Steps





In [14]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.naive_bayes import GaussianNB
# setup method for missing data using a median imputer for age and fair
numeric_features = ['Age', 'Fare']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))])

# setup one hot enoding for catagorical features
categorical_features = ['Embarked', 'Sex', 'Pclass']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# combine numeric and catagorical transformation into one column transformer object
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
pipe = Pipeline(steps=[('preprocessor', preprocessor), ('estimator',GaussianNB() )])
pipe.fit(data, y)

# drop rows where survived is unknown



Pipeline(memory=None,
     steps=[('preprocessor', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('num', Pipeline(memory=None,
     steps=[('imputer', SimpleImputer(copy=True, fill_value=None, missing_values=nan,
       strategy='median', verbo...)]), ['Embarked', 'Sex', 'Pclass'])])), ('estimator', GaussianNB(priors=None, var_smoothing=1e-09))])

# run cross validation on the the entire pipeline
# this does run the preprocessing transformation for every fold 


## Nieve Bayes Classifiers
https://www.datacamp.com/community/tutorials/naive-bayes-scikit-learn


In [15]:
# run basic cross validation 
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
scores = cross_val_score(pipe, data, y, cv=10, n_jobs=5, scoring='roc_auc')
print('all scores:', scores)
print(f'best auc score: {scores[np.argmax(scores)]}')


all scores: [0.94871795 0.8974359  0.94871795 0.93055556 0.54166667 0.88888889
 0.72222222 0.72222222 0.91666667 0.76388889]
best auc score: 0.9487179487179488


In [54]:
# get the keys
print(pipe.get_params().keys())

dict_keys(['memory', 'steps', 'preprocessor', 'estimator', 'preprocessor__n_jobs', 'preprocessor__remainder', 'preprocessor__sparse_threshold', 'preprocessor__transformer_weights', 'preprocessor__transformers', 'preprocessor__num', 'preprocessor__cat', 'preprocessor__num__memory', 'preprocessor__num__steps', 'preprocessor__num__imputer', 'preprocessor__num__imputer__copy', 'preprocessor__num__imputer__fill_value', 'preprocessor__num__imputer__missing_values', 'preprocessor__num__imputer__strategy', 'preprocessor__num__imputer__verbose', 'preprocessor__cat__memory', 'preprocessor__cat__steps', 'preprocessor__cat__imputer', 'preprocessor__cat__onehot', 'preprocessor__cat__imputer__copy', 'preprocessor__cat__imputer__fill_value', 'preprocessor__cat__imputer__missing_values', 'preprocessor__cat__imputer__strategy', 'preprocessor__cat__imputer__verbose', 'preprocessor__cat__onehot__categorical_features', 'preprocessor__cat__onehot__categories', 'preprocessor__cat__onehot__dtype', 'preproces

In [17]:
# tune parameters with csv search
from sklearn.model_selection import GridSearchCV

params = {'preprocessor__num__imputer__strategy': 
              ('mean', 'median', 'most_frequent', 'constant')}
model_gridcv = GridSearchCV(pipe, params, n_jobs=5, scoring='roc_auc', cv=10)
model_gridcv.fit(data, y)
model_gridcv.best_estimator_
model_gridcv.score(data, y)

c:\python36\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


0.8254742547425474